In [1537]:
import seaborn as sns
import pylab as rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib 
import pandas as pd 
import numpy as np
from tqdm.notebook import tqdm
import pytorch_lightning as pl
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [1538]:
sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))


tqdm.pandas()

In [1539]:
pl.seed_everything(42)

Seed set to 42


42

In [1540]:
df=pd.read_csv('/home/j/usfq/tesis/StockPredictionModels/Data/Complete.csv')
df

,Date,ticker_symbol,p_sentiment,Open,High,Low,Close,unrate,psr,m2,dspic,pce,reer,ir,ffer,tcs,indpro,ccpi
0,2015-01-02,AAPL,0.857156,27.847500,27.860001,26.837500,27.332500,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
1,2015-01-02,AMZN,0.731959,15.629000,15.737500,15.348000,15.426000,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
2,2015-01-02,GOOG,0.883473,26.378078,26.490770,26.133251,26.168653,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
3,2015-01-02,GOOGL,0.908735,26.629999,26.790001,26.393999,26.477501,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
4,2015-01-02,MSFT,0.804051,46.660000,47.419998,46.540001,46.759998,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7531,2019-12-30,AMZN,0.748412,93.699997,94.199997,92.030998,92.344498,3.6,7.3,15416.2,14844.1,14686.3,116.51,1.862857,1.550968,1458485.0,101.6179,265.651
7532,2019-12-30,GOOG,0.676151,67.500000,67.650002,66.700996,66.806999,3.6,7.3,15416.2,14844.1,14686.3,116.51,1.862857,1.550968,1458485.0,101.6179,265.651
7533,2019-12-30,GOOGL,0.705876,67.840500,67.849998,66.891998,66.985497,3.6,7.3,15416.2,14844.1,14686.3,116.51,1.862857,1.550968,1458485.0,101.6179,265.651
7534,2019-12-30,MSFT,0.762997,158.990005,159.020004,156.729996,157.589996,3.6,7.3,15416.2,14844.1,14686.3,116.51,1.862857,1.550968,1458485.0,101.6179,265.651


In [1541]:
#turn date into unix time
df['Date'] = pd.to_datetime(df['Date'])
df['Date'] = df['Date'].apply(lambda x: x.timestamp())
df
    

,Date,ticker_symbol,p_sentiment,Open,High,Low,Close,unrate,psr,m2,dspic,pce,reer,ir,ffer,tcs,indpro,ccpi
0,1.420157e+09,AAPL,0.857156,27.847500,27.860001,26.837500,27.332500,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
1,1.420157e+09,AMZN,0.731959,15.629000,15.737500,15.348000,15.426000,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
2,1.420157e+09,GOOG,0.883473,26.378078,26.490770,26.133251,26.168653,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
3,1.420157e+09,GOOGL,0.908735,26.629999,26.790001,26.393999,26.477501,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
4,1.420157e+09,MSFT,0.804051,46.660000,47.419998,46.540001,46.759998,5.7,8.0,11759.1,13224.7,12036.5,106.11,1.881500,0.114839,1069010.0,102.8479,239.811
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7531,1.577664e+09,AMZN,0.748412,93.699997,94.199997,92.030998,92.344498,3.6,7.3,15416.2,14844.1,14686.3,116.51,1.862857,1.550968,1458485.0,101.6179,265.651
7532,1.577664e+09,GOOG,0.676151,67.500000,67.650002,66.700996,66.806999,3.6,7.3,15416.2,14844.1,14686.3,116.51,1.862857,1.550968,1458485.0,101.6179,265.651
7533,1.577664e+09,GOOGL,0.705876,67.840500,67.849998,66.891998,66.985497,3.6,7.3,15416.2,14844.1,14686.3,116.51,1.862857,1.550968,1458485.0,101.6179,265.651
7534,1.577664e+09,MSFT,0.762997,158.990005,159.020004,156.729996,157.589996,3.6,7.3,15416.2,14844.1,14686.3,116.51,1.862857,1.550968,1458485.0,101.6179,265.651


In [1542]:
#only use date, open, high, low, close
#df = df[['Date', 'ticker_symbol','p_sentiment','unrate','m2','Open', 'High', 'Low', 'Close']]

In [1543]:
#generate new dataframes for each ticker_symbol
df_dict={}
for key in df['ticker_symbol'].unique():
    df_dict[key]=df[df['ticker_symbol']==key]
    df_dict[key]=df_dict[key].drop(columns=['ticker_symbol'])
    df_dict[key]=df_dict[key].sort_values(by=['Date']).reset_index(drop=True)
    df_dict[key]=df_dict[key].drop(columns=['Date'])
    print(key,df_dict[key].shape)
    print(df_dict[key].head(3))

AAPL (1252, 16)
   p_sentiment     Open       High        Low      Close  unrate  psr  \
0     0.857156  27.8475  27.860001  26.837500  27.332500     5.7  8.0   
1     0.744531  27.0725  27.162500  26.352501  26.562500     5.7  8.0   
2     0.769826  26.6350  26.857500  26.157499  26.565001     5.7  8.0   

        m2    dspic      pce    reer      ir      ffer        tcs    indpro  \
0  11759.1  13224.7  12036.5  106.11  1.8815  0.114839  1069010.0  102.8479   
1  11759.1  13224.7  12036.5  106.11  1.8815  0.114839  1069010.0  102.8479   
2  11759.1  13224.7  12036.5  106.11  1.8815  0.114839  1069010.0  102.8479   

      ccpi  
0  239.811  
1  239.811  
2  239.811  
AMZN (1257, 16)
   p_sentiment     Open     High      Low    Close  unrate  psr       m2  \
0     0.731959  15.6290  15.7375  15.3480  15.4260     5.7  8.0  11759.1   
1     0.696571  15.3505  15.4190  15.0425  15.1095     5.7  8.0  11759.1   
2     0.640295  15.1120  15.1500  14.6190  14.7645     5.7  8.0  11759.1   

 

In [1544]:
train_df_dict={}
test_df_dict={}
val_df_dict={}
for key in df_dict.keys():
    train_size=int(len(df_dict[key])*0.9)
    train_df_dict[key], test_df_dict[key]=df_dict[key][:train_size], df_dict[key][train_size:]
    train_df_dict[key],val_df_dict[key]=train_df_dict[key][:int(len(train_df_dict[key])-126)],train_df_dict[key][int(len(train_df_dict[key])-126):]
    print(val_df_dict[key].shape, train_df_dict[key].shape, test_df_dict[key].shape)


(126, 16) (1000, 16) (126, 16)
(126, 16) (1005, 16) (126, 16)
(126, 16) (1004, 16) (126, 16)
(126, 16) (1005, 16) (126, 16)
(126, 16) (1005, 16) (126, 16)
(126, 16) (1005, 16) (126, 16)


In [1545]:
ticker='AAPL'

In [1546]:
train_df=train_df_dict[ticker]
test_df=test_df_dict[ticker]
val_df=val_df_dict[ticker]

In [1547]:
scaler=MinMaxScaler(feature_range=(0,1))
scaler=scaler.fit(train_df)

In [1548]:
train_df=pd.DataFrame(scaler.transform(train_df), columns=train_df.columns, index=train_df.index)
test_df=pd.DataFrame(scaler.transform(test_df), columns=test_df.columns, index=test_df.index)
val_df=pd.DataFrame(scaler.transform(val_df), columns=val_df.columns, index=val_df.index)
train_df.head()

,p_sentiment,Open,High,Low,Close,unrate,psr,m2,dspic,pce,reer,ir,ffer,tcs,indpro,ccpi
0,0.950398,0.151939,0.139422,0.127432,0.133987,1.0,0.6,0.0,0.0,0.0,0.0,0.229028,0.001906,0.0,0.779248,0.0
1,0.675767,0.129919,0.119746,0.113606,0.112256,1.0,0.6,0.0,0.0,0.0,0.0,0.229028,0.001906,0.0,0.779248,0.0
2,0.737447,0.117488,0.111142,0.108046,0.112326,1.0,0.6,0.0,0.0,0.0,0.0,0.229028,0.001906,0.0,0.779248,0.0
3,0.747614,0.122176,0.116573,0.122799,0.122839,1.0,0.6,0.0,0.0,0.0,0.0,0.229028,0.001906,0.0,0.779248,0.0
4,0.738988,0.136596,0.144429,0.137054,0.152050,1.0,0.6,0.0,0.0,0.0,0.0,0.229028,0.001906,0.0,0.779248,0.0


In [1549]:
def create_sequences(input_data: pd.DataFrame, target_column, sequence_length):
    sequences=[]
    data_size=len(input_data)

    for i in tqdm(range(data_size-sequence_length)):
        sequence=input_data[i:i+sequence_length].copy()
        label_position=i+sequence_length
        label=input_data.iloc[label_position][target_column]
        sequences.append((sequence, label))
    return sequences


In [1550]:
SEQUENCE_LENGTH=14
#consider changing to high and low
train_sequences=create_sequences(train_df, 'Close', SEQUENCE_LENGTH)
test_sequences=create_sequences(test_df, 'Close', SEQUENCE_LENGTH)
val_sequences=create_sequences(val_df, 'Close', SEQUENCE_LENGTH)

  0%|          | 0/986 [00:00<?, ?it/s]

  0%|          | 0/112 [00:00<?, ?it/s]

  0%|          | 0/112 [00:00<?, ?it/s]

PyTorch


In [1551]:
class StockDataset(Dataset):
    def __init__(self, sequences):
        self.sequences=sequences

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        sequence, label=self.sequences[index]
        return dict(
            sequence=torch.Tensor(sequence.to_numpy()),
            label=torch.Tensor([label]).float()
        )

In [1552]:
class StockDataModule(pl.LightningDataModule):
    def __init__(self, train_sequences, test_sequences, val_sequences, batch_size=128):
        super().__init__()
        self.train_sequences=train_sequences
        self.test_sequences=test_sequences
        self.val_sequences=val_sequences
        self.batch_size=batch_size

    def setup(self,stage=None):
        self.train_dataset=StockDataset(self.train_sequences)
        self.test_dataset=StockDataset(self.test_sequences)
        self.val_dataset=StockDataset(self.val_sequences)

    def train_dataloader(self): #considerar el shuffle
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True,num_workers=2)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=1, shuffle=False,num_workers=1)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=1, shuffle=False,num_workers=1)

In [1553]:
N_EPOCHS=1024
BATCH_SIZE=256

data_module=StockDataModule(train_sequences, test_sequences, val_sequences, batch_size=BATCH_SIZE)
data_module.setup()

In [1554]:
train_dataset=StockDataset(train_sequences)

In [1555]:
for item in train_dataset:
    print(item['sequence'].shape)
    print(item['label'].shape)
    print(item['label'])
    break

torch.Size([14, 16])
torch.Size([1])
tensor([0.1597])


#MODEL

In [1556]:
class LSTM_Model(nn.Module):
    def __init__(self, n_features, n_hidden=128, n_layers=2):
        super().__init__()
        self.n_hidden=n_hidden
        self.n_layers=n_layers
        self.lstm=nn.LSTM(input_size=n_features, hidden_size=n_hidden, num_layers=n_layers, batch_first=True,)
        self.linear=nn.Linear(in_features=n_hidden, out_features=1) #considerar cambiar a 2

    def forward(self,x):
        self.lstm.flatten_parameters()
        _, (hidden, _)= self.lstm(x)
        out= hidden[-1]

        return self.linear(out)
        

In [1557]:
class StockLSTM(pl.LightningModule):
    def __init__(self, n_features):
        super().__init__()
        self.criterion=nn.MSELoss()
        self.model=LSTM_Model(n_features)

    def forward(self, x, labels=None):
        output= self.model(x)
        loss=0
        if labels is not None:
            loss=self.criterion(output, labels.unsqueeze(dim=1))
        return loss, output

    def training_step(self, batch, batch_idx):
        x=batch['sequence']
        y=batch['label']
        loss, outputs= self(x, y)
        self.log('train_loss', loss,prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=0.0001)

    def validation_step(self, batch, batch_idx):
        x=batch['sequence']
        y=batch['label']
        loss, outputs= self(x, y)
        self.log('val_loss', loss,prog_bar=True, logger=True)
        return  loss

    def test_step(self, batch, batch_idx):
        x=batch['sequence']
        y=batch['label']
        loss, outputs= self(x, y)
        self.log('test_loss', loss,prog_bar=True, logger=True)
        return loss

    def predict(self, x, y):
        y_hat=self(x).squeeze()
        loss=self.loss(y_hat, y)
        return y_hat, loss

    def save_model(self, path):
        torch.save(self.state_dict(), path)

    def load_model(self, path):
        self.load_state_dict(torch.load(path))

In [1558]:
train_df.shape

(1000, 16)

In [1559]:
model = StockLSTM(n_features=train_df.shape[1])

In [1560]:
for item in data_module.train_dataloader():
    print(item['sequence'].shape)
    print(item['label'].shape)
    #print(item['label'])
    break

torch.Size([256, 14, 16])
torch.Size([256, 1])


In [1561]:
checkpoint_callback=pl.callbacks.ModelCheckpoint(
    dirpath='checkpoints',
    filename='best-checkpoint',
    save_top_k=3,
    verbose=True,
    monitor='val_loss',
    mode='min'
)

logger=pl.loggers.TensorBoardLogger('lightning_logs/', name='stock-lstm')

early_stopping_callback=pl.callbacks.EarlyStopping(monitor='val_loss', patience=3,verbose=False,mode='min')

trainer = pl.Trainer(
    logger=logger,
    callbacks=[ checkpoint_callback,early_stopping_callback],
    max_epochs=N_EPOCHS,
)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [1562]:
trainer.fit(model, data_module)

Missing logger folder: lightning_logs/stock-lstm
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | criterion | MSELoss    | 0     
1 | model     | LSTM_Model | 206 K 
-----------------------------------------
206 K     Trainable params
0         Non-trainable params
206 K     Total params
0.828     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/j/miniconda3/envs/gen/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.
/home/j/miniconda3/envs/gen/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 1, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/j/miniconda3/envs/gen/lib/python3.9/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

/home/j/miniconda3/envs/gen/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([256, 1, 1])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/j/miniconda3/envs/gen/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([218, 1, 1])) that is different to the input size (torch.Size([218, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

Epoch 12, global step 52: 'val_loss' reached 0.05887 (best 0.05107), saving model to '/home/j/usfq/tesis/StockPredictionModels/Models/LSTM/checkpoints/best-checkpoint.ckpt' as top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 13, global step 56: 'val_loss' was not in top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 14, global step 60: 'val_loss' was not in top 3


In [1563]:
trained_model=StockLSTM.load_from_checkpoint('checkpoints/best-checkpoint-v1.ckpt', n_features=test_df.shape[1])

In [1564]:
trained_model.freeze()

In [1565]:
test_dataset=StockDataset(test_sequences)

predictions = []
labels=[]

trained_model.to('cpu') # move the model to the CPU

for item in tqdm(test_dataset):
    x=item['sequence']
    y=item['label']
   
    _, output= trained_model(x.unsqueeze(dim=0)) # send input to the device
    predictions.append(output.item())
    labels.append(y.item())

  0%|          | 0/112 [00:00<?, ?it/s]

In [1566]:
len(predictions),len(test_dataset)

(112, 112)

In [1567]:
descaler=MinMaxScaler(feature_range=(0,1))
descaler.min_,descaler.scale_=scaler.min_[-1],scaler.scale_[-1]

In [1568]:
def descale(descaler,values):
    values_2d=np.array(values)[:, np.newaxis]
    return descaler.inverse_transform(values_2d).flatten()

In [1569]:
i = 1  # for the second column as an example
feature_min = scaler.data_min_[4]
feature_max = scaler.data_max_[4]

In [1570]:
def denormalize_feature(feature_normalized, feature_min, feature_max):
    return feature_normalized * (feature_max - feature_min) + feature_min

In [1571]:
predictions = denormalize_feature(np.array(predictions), feature_min, feature_max)
labels = denormalize_feature(np.array(labels), feature_min, feature_max)

In [1572]:
%load_ext tensorboard
%tensorboard --logdir ./lightning_logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 84465), started 2:04:29 ago. (Use '!kill 84465' to kill it.)